In [1]:
from pathlib import Path
import sys, site, platform, os, subprocess,time, json
import pandas as pd

In [2]:
# cargar dotenv (instala python-dotenv si hace falta)
try:
    from dotenv import load_dotenv
    load_dotenv()   # carga .env desde el working dir o sus padres
except Exception:
    pass

secret = os.getenv("OPENAI_API_KEY")
if not secret:
    raise RuntimeError(
        "OPENAI_API_KEY no encontrada. Verifica .env en la ruta y que tenga formato: OPENAI_API_KEY=sk-... (sin comillas)"
    )

# No imprimir la clave. Verificar solo que se cargó:
print("OPENAI_API_KEY cargada correctamente:", bool(secret))

OPENAI_API_KEY cargada correctamente: True


In [3]:
print("Python exe:", sys.executable)
print("Version:", sys.version)
print("Site-packages:", site.getsitepackages() if hasattr(site,"getsitepackages") else site.getusersitepackages())
print("OS:", platform.platform())

Python exe: c:\Users\aleja\anaconda3\envs\bi\python.exe
Version: 3.12.9 | packaged by conda-forge | (main, Mar  4 2025, 22:37:18) [MSC v.1943 64 bit (AMD64)]
Site-packages: ['c:\\Users\\aleja\\anaconda3\\envs\\bi', 'c:\\Users\\aleja\\anaconda3\\envs\\bi\\Lib\\site-packages']
OS: Windows-11-10.0.26100-SP0


In [4]:
from openai import OpenAI
client = OpenAI(api_key=secret)
models = client.models.list()
print("OK SDK v1.x, modelos:", len(models.data))

OK SDK v1.x, modelos: 96


In [5]:
TEXTO = "textos"  # columna de la opinión
ODS = "labels"  # columna de etiqueta 1,3,4

df = pd.read_excel(r"datos/Datos_proyecto.xlsx")
ods_list = sorted(df[ODS].unique())
print("ODS encontrados:", ods_list)

ODS encontrados: [1, 3, 4]


In [6]:
def generar_opiniones(ods_id, ejemplos_ref, n=30):
    """Genera n ejemplos sintéticos para un ODS específico"""
    ejemplos = "\n".join(f"- {s}" for s in ejemplos_ref)

    prompt = f"""
    Genera {n} opiniones ciudadanas breves (1–2 oraciones), en español de Colombia,
    realistas y respetuosas, sobre problemáticas locales mapeadas SOLO al ODS {ods_id}.
    Definición de cada ODS:
    ODS 1: Fin de la pobreza, ODS 3: Salud y Bienestar, ODS 4: Educación de calidad.

    Requisitos:
    - TODAS deben corresponder al ODS {ods_id}.
    - Varía zonas (urbano/rural), actores e instituciones; evita datos personales.
    - Mantén neutralidad política y sin contenido sensible.
    - Entrega SOLO JSON válido: una lista de objetos con:
      "textos" (string) y "labels" (entero {ods_id}).

    Ejemplos reales del dataset (NO copiar literalmente):
    {ejemplos if ejemplos_ref else '- (sin ejemplos de contexto)'}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un generador de datos sintéticos."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
    )

    txt = response.choices[0].message.content.strip()
    if txt.startswith("```"):
        txt = txt.strip("`")
        if "\n" in txt:
            txt = txt.split("\n", 1)[1]

    data = json.loads(txt)
    df_new = pd.DataFrame(data)
    df_new["labels"] = ods_id

    print(f"ODS {ods_id}: {len(df_new)} nuevos ejemplos")
    print(f"Tokens usados: prompt={response.usage.prompt_tokens}, completion={response.usage.completion_tokens}, total={response.usage.total_tokens}")
    return df_new


# Generar 5 datasets sintéticos diferentes
datasets = []
for i in range(1, 6):
    print(f"\n--- Generando dataset sintético #{i} ---")

    df_sintetico = pd.DataFrame(columns=[TEXTO, ODS])
    for ods_id in ods_list:
        # Muestras de referencia de ese ODS
        ejemplos_ref = (
            df[df[ODS] == ods_id][TEXTO]
            .dropna()
            .astype(str)
            .sample(min(8, sum(df[ODS] == ods_id)), random_state=42)
            .tolist()
        )

        nuevos = generar_opiniones(ods_id, ejemplos_ref, n=30)
        df_sintetico = pd.concat([df_sintetico, nuevos], ignore_index=True)

    df_sintetico = df_sintetico.sample(frac=1, random_state=i).reset_index(drop=True)
    datasets.append(df_sintetico)
    print(f"Dataset sintético {i} → {len(df_sintetico)} filas")


--- Generando dataset sintético #1 ---
ODS 1: 32 nuevos ejemplos
Tokens usados: prompt=1205, completion=1382, total=2587
ODS 3: 32 nuevos ejemplos
Tokens usados: prompt=1455, completion=1122, total=2577
ODS 4: 31 nuevos ejemplos
Tokens usados: prompt=1218, completion=1152, total=2370
Dataset sintético 1 → 95 filas

--- Generando dataset sintético #2 ---
ODS 1: 31 nuevos ejemplos
Tokens usados: prompt=1205, completion=1306, total=2511
ODS 3: 32 nuevos ejemplos
Tokens usados: prompt=1455, completion=1216, total=2671
ODS 4: 28 nuevos ejemplos
Tokens usados: prompt=1218, completion=1168, total=2386
Dataset sintético 2 → 91 filas

--- Generando dataset sintético #3 ---
ODS 1: 30 nuevos ejemplos
Tokens usados: prompt=1205, completion=1346, total=2551
ODS 3: 30 nuevos ejemplos
Tokens usados: prompt=1455, completion=1109, total=2564
ODS 4: 31 nuevos ejemplos
Tokens usados: prompt=1218, completion=1272, total=2490
Dataset sintético 3 → 91 filas

--- Generando dataset sintético #4 ---
ODS 1: 30

In [7]:
for i, df_syn in enumerate(datasets, start=1):
    df_syn.to_excel(f"datos_pruebas_sintenticos/Datos_sinteticos_{i}.xlsx", index=False)

In [ ]:
df_ref = pd.read_excel(r"datos/Datos de prueba_proyecto.xlsx")

# Toma una muestra representativa para dar contexto al modelo
ejemplos_ref = df_ref["Textos_espanol"].sample(min(10, len(df_ref)), random_state=42).tolist()
ejemplos = "\n".join(f"- {s}" for s in ejemplos_ref)


def generar_textos_similares(ejemplos_ref, n=100):
    """Genera n textos similares al archivo de referencia (sin etiquetas)"""
    prompt = f"""
    Tienes ejemplos de opiniones ciudadanas reales en español de Colombia.

    Genera {n} nuevos textos que se parezcan en tono, extensión y estilo a estos ejemplos,
    pero con contenido totalmente nuevo (no repetir ni copiar frases).
    Las opiniones deben sonar naturales y variadas, sobre situaciones cotidianas
    que podría mencionar una persona en su comunidad.

    Ejemplos de referencia:
    {ejemplos}

    Entrega la respuesta en formato JSON válido, con una lista de objetos:
    [{{"textos": "opinión..." }}, ...]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Eres un generador de textos ciudadanos realistas."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.8,
    )

    txt = response.choices[0].message.content.strip()
    if txt.startswith("```"):
        txt = txt.strip("`")
        if "\n" in txt:
            txt = txt.split("\n", 1)[1]

    data = json.loads(txt)
    df_new = pd.DataFrame(data)

    print(f"{len(df_new)} textos generados.")
    print(f"Tokens usados: prompt={response.usage.prompt_tokens}, completion={response.usage.completion_tokens}, total={response.usage.total_tokens}")
    return df_new


# 🔁 Crear varios archivos de prueba similares
n_archivos = 5
textos_por_archivo = 120
datasets_no_label = []

for i in range(1, n_archivos + 1):
    print(f"\n--- Generando archivo de prueba #{i} ---")
    df_fake = generar_textos_similares(ejemplos_ref, n=textos_por_archivo)
    datasets_no_label.append(df_fake)
    df_fake.to_excel(f"datos_pruebas_sintenticos/Textos_prueba_{i}.xlsx", index=False)

print("Archivos generados y guardados.")


--- Generando archivo de prueba #1 ---
73 textos generados.
Tokens usados: prompt=1384, completion=3190, total=4574

--- Generando archivo de prueba #2 ---
63 textos generados.
Tokens usados: prompt=1384, completion=2554, total=3938

--- Generando archivo de prueba #3 ---
76 textos generados.
Tokens usados: prompt=1384, completion=3144, total=4528

--- Generando archivo de prueba #4 ---
66 textos generados.
Tokens usados: prompt=1384, completion=2739, total=4123

--- Generando archivo de prueba #5 ---
85 textos generados.
Tokens usados: prompt=1384, completion=2807, total=4191
✅ Archivos generados y guardados.
